In [1]:
!pip -q install shapely pyproj fiona rasterio folium branca ipywidgets requests  scipy
!pip install requests networkx pandas geopandas
!pip install --upgrade pyvis langchain langchain-google-genai
!pip install matplotlib ipykernel ipympl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 15.5 MB/s eta 0:00:00


In [2]:
import os, sys, io, json, math, time, random, uuid, calendar, traceback, re
from dataclasses import dataclass
from typing import TypedDict, Dict, Any, Optional, List, Tuple

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
from io import StringIO

import folium
from folium.plugins import Fullscreen, MiniMap
from shapely.geometry import box, mapping

from langgraph.types import Command
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

print('Python', sys.version.split()[0])
print('pandas', pd.__version__, 'geopandas', gpd.__version__, 'folium', folium.__version__)

Python 3.12.12
pandas 2.2.2 geopandas 1.1.2 folium 0.20.0


In [3]:
# ==== PERSISTENT OUTPUT CONFIG (Google Drive) ====
from google.colab import drive
import os, time, shutil
from pathlib import Path

# 1) Mount Drive
drive.mount('/content/drive')

# 2) Base folder (the one you requested)
BASE_DIR = Path("/content/drive/MyDrive/AI/langgraph_multi_agent")
BASE_DIR.mkdir(parents=True, exist_ok=True)

# 3) Put all outputs under a single subfolder (recommended)
OUT_DIR = BASE_DIR / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 4) Artifact folder (HTML/PNG required names go here)
ARTIFACT_DIR = OUT_DIR / "artifacts"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

# 5) Logs / tables
LOG_DIR = OUT_DIR / "logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)

RUNS_CSV_PATH = LOG_DIR / "runs.csv"
COST_CSV_PATH = LOG_DIR / "codegen_cost.csv"

print("BASE_DIR:", BASE_DIR)
print("OUT_DIR:", OUT_DIR)
print("ARTIFACT_DIR:", ARTIFACT_DIR)
print("RUNS_CSV_PATH:", RUNS_CSV_PATH)
print("COST_CSV_PATH:", COST_CSV_PATH)


Mounted at /content/drive
BASE_DIR: /content/drive/MyDrive/AI/langgraph_multi_agent
OUT_DIR: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs
ARTIFACT_DIR: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs/artifacts
RUNS_CSV_PATH: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs/logs/runs.csv
COST_CSV_PATH: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs/logs/codegen_cost.csv


In [4]:
# ===== PATCH 2: robust HTTP download helpers (retry/backoff + timeouts) =====
import os, time, random
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def _requests_session():
    s = requests.Session()
    retry = Retry(
        total=6,
        connect=6,
        read=6,
        backoff_factor=1.0,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "HEAD"]
    )
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

def download_file(url: str, out_path: str, timeout=(10, 60)) -> str:
    """
    Download to local disk reliably. Returns out_path.
    timeout = (connect_timeout, read_timeout)
    """
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        return out_path

    s = _requests_session()
    with s.get(url, stream=True, timeout=timeout) as r:
        r.raise_for_status()
        tmp = out_path + ".part"
        with open(tmp, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    f.write(chunk)
        os.replace(tmp, out_path)
    return out_path


In [5]:
from google.colab import userdata

key = userdata.get('GOOGLE_API_KEY')
if not key:
    raise RuntimeError('Không đọc được secret GOOGLE_API_KEY. Vào Colab → Secrets → tạo GOOGLE_API_KEY và bật Notebook access, rồi Restart runtime.')

os.environ['GOOGLE_API_KEY'] = key
print('Loaded GOOGLE_API_KEY from Colab Secrets.')


Loaded GOOGLE_API_KEY from Colab Secrets.


In [6]:
MODEL_NAME = 'gemini-2.5-flash-lite'
TEMPERATURE = 0.0

llm = ChatGoogleGenerativeAI(
    model=MODEL_NAME,
    google_api_key=os.environ['GOOGLE_API_KEY'],
    temperature=TEMPERATURE,
)

_MIN_SECONDS_BETWEEN_CALLS = 8.0
_last_call_t = 0.0

@dataclass
class LLMUsage:
    api_calls: int = 0
    prompt_chars: int = 0
    completion_chars: int = 0
    prompt_tokens: int = 0
    completion_tokens: int = 0

USAGE = LLMUsage()

def _estimate_tokens_from_text(s: str) -> int:
    return int(max(1, len(s) / 4))

def _throttle():
    global _last_call_t
    now = time.time()
    wait = _MIN_SECONDS_BETWEEN_CALLS - (now - _last_call_t)
    if wait > 0:
        time.sleep(wait)
    _last_call_t = time.time()

def call_llm(system_prompt: str, user_prompt: str, max_tries: int = 8):
    USAGE.prompt_chars += len(system_prompt) + len(user_prompt)

    for i in range(max_tries):
        try:
            _throttle()
            USAGE.api_calls += 1
            resp = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=user_prompt)])
            text = resp.content or ''
            USAGE.completion_chars += len(text)

            # Token usage: proxy (reproducible) — chars/4
            USAGE.prompt_tokens += _estimate_tokens_from_text(system_prompt + user_prompt)
            USAGE.completion_tokens += _estimate_tokens_from_text(text)

            meta = getattr(resp, 'response_metadata', {}) or {}
            return text, meta
        except Exception as e:
            emsg = str(e)
            is_quota = ('429' in emsg) or ('RESOURCE_EXHAUSTED' in emsg) or ('rate limit' in emsg.lower())
            if is_quota and i < max_tries - 1:
                sleep_s = min(120.0, (2 ** i) * 2.0 + random.uniform(0.0, 1.0))
                print(f'[LLM] quota/429 -> backoff {sleep_s:.1f}s (try {i+1}/{max_tries})')
                time.sleep(sleep_s)
                continue
            raise

print('Gemini model:', MODEL_NAME, '| temperature:', TEMPERATURE)

Gemini model: gemini-2.5-flash-lite | temperature: 0.0


In [7]:
def try_get(url, timeout=15):
    try:
        r = requests.get(url, timeout=timeout)
        if r.status_code == 200:
            return r
    except Exception:
        pass
    return None

CASE_SPECS = {
    'france_mobility': {'required_files': ['france_mobility_map_matrix.png','france_mobility_line.png','france_mobility_dec2020.html']},
    'hazardous_nc': {'required_files': ['nc_hazardous_map.html','nc_population_hist.png']},
    'covid_us': {'required_files': ['covid_death_rate_map.html','covid_senior_scatter.png']},
}

def artifacts_ok(case: str) -> bool:
    import os
    req = CASE_SPECS[case]['required_files']
    missing = [f for f in req if not os.path.exists(f)]
    if missing:
        print('[validator] missing files:', missing)
        return False
    return True

In [8]:
def code_template_france() -> str:
    return r'''
# === France Mobility 2020 ===
import sys
import numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt, calendar
from shapely.geometry import box, mapping
import folium
from folium.plugins import Fullscreen, MiniMap
from io import StringIO

# Dữ liệu ranh giới
fr_url = "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip"
try:
    local_zip = "France.zip"
    download_file(fr_url, local_zip)
    france_gdf = gpd.read_file(f"zip://{local_zip}")
    src_desc = "local France.zip"
except Exception:
    # synthetic fallback: 4 vùng vuông
    polys = [box(i*2, j*2, i*2+1.8, j*2+1.5) for i in range(2) for j in range(2)]
    france_gdf = gpd.GeoDataFrame({"GID_1":[f"FRA.{i+1}_1" for i in range(4)],
                                   "NAME_1":[f"R{i+1}" for i in range(4)]},
                                  geometry=polys, crs="EPSG:4326")
    src_desc = "synthetic 4 regions"

# Dữ liệu mobility (Twitter API) hoặc synthetic
api = "http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020"
r = None  # FORCE SYNTHETIC (paper/Colab stability: avoid external API dependency)
if r is not None:
    mob_df = pd.read_csv(StringIO(r.text))
    if "date" in mob_df.columns:
        mob_df["date"] = pd.to_datetime(mob_df["date"], errors="coerce")
        mob_df = mob_df.dropna(subset=["date"])
        mob_df["month"] = mob_df["date"].dt.month
        mob_df = mob_df.groupby(["place","month"])["intra_movement"].sum().reset_index()
else:
    # synthetic monthly pattern
    rows = []
    np.random.seed(42)
    for pl in france_gdf["GID_1"].astype(str).tolist():
        base = np.random.randint(1000,5000)
        for m in range(1,13):
            factor = 0.8 + 0.4*np.sin((m-1)*np.pi/6)  # dùng numpy thay math
            if 3<=m<=5: factor *= 0.5
            val = int(base*factor*np.random.uniform(0.9,1.1))
            rows.append({"place": pl, "month": m, "intra_movement": val})
    mob_df = pd.DataFrame(rows)
# --- schema normalization (robust across France.zip variants) ---
cols = set(france_gdf.columns)

# try to find id/name columns
gid_candidates  = [c for c in ["GID_1","GID","GID1","ID_1","ID"] if c in cols]
name_candidates = [c for c in ["NAME_1","NAME","NAME1","NAME_EN","region","Region"] if c in cols]

if "GID_1" not in cols:
    if gid_candidates:
        france_gdf["GID_1"] = france_gdf[gid_candidates[0]].astype(str)
    else:
        france_gdf["GID_1"] = france_gdf.index.astype(str)

if "NAME_1" not in cols:
    if name_candidates:
        france_gdf["NAME_1"] = france_gdf[name_candidates[0]].astype(str)
    else:
        france_gdf["NAME_1"] = france_gdf["GID_1"].astype(str)

# Pivot + change vs January
piv = mob_df.pivot(index="place", columns="month", values="intra_movement")
piv = piv.rename(columns={i: calendar.month_name[i] for i in range(1,13)})
piv = piv.fillna(0.0)
jan = piv["January"].replace(0, np.nan)
for m in range(2,13):
    mm = calendar.month_name[m]
    piv[f"{mm}_change"] = (piv[mm] / jan - 1.0) * 100.0
piv = piv.replace([np.inf,-np.inf], np.nan).fillna(0.0)
change_df = piv.reset_index()  # 'place' + *_change

fr = france_gdf.copy()
fr["GID_1"] = fr["GID_1"].astype(str)
change_df["place"] = change_df["place"].astype(str)
fr_merge = fr.merge(change_df, left_on="GID_1", right_on="place", how="inner")
if fr_merge.empty:
    fr["place"] = fr["GID_1"]
    fr_merge = fr.merge(change_df, on="place", how="inner")

# ===== 1) Ma trận 12 tháng (matplotlib) — dùng constrained_layout thay tight_layout
fr_plot = fr_merge.to_crs(epsg=3857)
vals = []
for m in range(2,13):
    col = f"{calendar.month_name[m]}_change"
    if col in fr_plot.columns:
        vals.extend(pd.to_numeric(fr_plot[col], errors="coerce").astype(float).tolist())
vmax = float(np.nanpercentile(vals, 98)) if len(vals) else 50.0
vmin = -vmax
cmap = plt.cm.coolwarm
fig, axes = plt.subplots(3,4, figsize=(20,15), constrained_layout=True); axes = axes.flatten()
for i, mth in enumerate(range(1,13)):
    ax = axes[i]
    if mth == 1:
        fr_plot.plot(ax=ax, color="lightgrey", edgecolor="white")
        ax.set_title("January (base)")
    else:
        col = f"{calendar.month_name[mth]}_change"
        fr_plot.plot(column=col, ax=ax, cmap=cmap, vmin=vmin, vmax=vmax, edgecolor="white", legend=False)
        ax.set_title(f"{calendar.month_name[mth]}")
    ax.set_axis_off()
cax = fig.add_axes([0.92, 0.1, 0.02, 0.8])
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax)); sm._A = []
fig.colorbar(sm, cax=cax).set_label("Monthly change vs Jan (%)")
fig.suptitle("France mobility changes 2020", fontsize=16)
fig_path = "france_mobility_map_matrix.png"
fig.savefig(fig_path, dpi=220, bbox_inches="tight"); plt.close(fig)

# ===== 2) Line chart (mean of regions)
monthly_cols = [f"{calendar.month_name[m]}_change" for m in range(2,13)]
month_mean = fr_merge[monthly_cols].astype(float).mean(axis=0)
plt.figure(figsize=(9,5))
plt.plot(monthly_cols, month_mean.values, marker="o")
plt.xticks(rotation=45); plt.grid(True, alpha=0.3)
plt.title("France mobility — average monthly change vs Jan")
line_path = "france_mobility_line.png"
plt.savefig(line_path, dpi=200, bbox_inches="tight"); plt.close()

# ===== 3) Folium map (December) — center bằng total_bounds (không centroid) + tô sẵn màu _fill
fr_map = fr_merge.to_crs("EPSG:4326").copy()
fr_map["GID_1"] = fr_map["GID_1"].astype(str)
if "December_change" not in fr_map.columns:
    fr_map["December_change"] = 0.0

# center an toàn (không dùng centroid trên CRS địa lý)
minx, miny, maxx, maxy = fr_map.total_bounds
center_lat, center_lon = (miny+maxy)/2.0, (minx+maxx)/2.0
m = folium.Map(location=[center_lat, center_lon], zoom_start=5, tiles="CartoDB positron")
Fullscreen().add_to(m); MiniMap(toggle_display=True, minimized=True).add_to(m)

# === Thang màu robust ===
vals_dec = pd.to_numeric(fr_map["December_change"], errors="coerce")
vals_dec = vals_dec.replace([np.inf, -np.inf], np.nan)

finite = vals_dec.dropna()
if len(finite) >= 2:
    lo, hi = np.nanpercentile(finite, [2, 98])
    # nếu percentile suy biến, fallback min/max
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        lo, hi = float(finite.min()), float(finite.max())
else:
    lo, hi = -50.0, 50.0  # fallback hợp lý

# tuỳ chọn: cân tâm 0 khi dữ liệu có cả âm & dương
if lo < 0 and hi > 0:
    mabs = max(abs(lo), abs(hi))
    lo, hi = -mabs, mabs

# chuẩn hoá về [0,1]
den = (hi - lo) if (hi - lo) != 0 else 1e-9
norm = (vals_dec - lo) / den
norm = norm.clip(0, 1)

# rgba -> hex bằng matplotlib colormap (không dùng cm trong lambda)
def _to_hex01(x):
    r, g, b, a = plt.cm.coolwarm(float(x))
    return "#{:02x}{:02x}{:02x}".format(int(r*255), int(g*255), int(b*255))

fr_map["_fill"] = [
    "#cccccc" if pd.isna(v) else _to_hex01(n)
    for v, n in zip(vals_dec.tolist(), norm.tolist())
]

# GeoJSON chuỗi (bao gồm cột _fill)
geojson_str = fr_map.to_json()

# Layer tô màu đọc từ thuộc tính _fill (KHÔNG dùng biến ngoài)
folium.GeoJson(
    geojson_str,
    name="December change",
    style_function=lambda f: {
        "fillColor": f["properties"].get("_fill", "#cccccc"),
        "color": "white", "weight": 0.4, "fillOpacity": 0.7
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NAME_1","GID_1","December_change"],
        aliases=["Region","GID","Δ vs Jan (%)"],
        localize=True, labels=True, sticky=False
    )
).add_to(m)

# Legend độc lập (không dùng trong lambda, chỉ để hiển thị)
from branca.colormap import LinearColormap
cm_legend = LinearColormap(["#313695","#74add1","#ffffbf","#f46d43","#a50026"], vmin=lo, vmax=hi)
cm_legend.caption = "Change vs Jan (%)"
cm_legend.add_to(m)

folium.LayerControl().add_to(m)

map_path = "france_mobility_dec2020.html"; m.save(map_path)

# ===== Tables & artifacts
table_regions = fr_merge[["NAME_1","GID_1"]+monthly_cols].copy()
metrics = pd.DataFrame({
    "metric": ["min_change","max_change","mean_december"],
    "value": [float(np.nanmin(month_mean)), float(np.nanmax(month_mean)), float(np.nanmean(vals_dec))]
})

ARTIFACTS = {
    "src_desc": src_desc,
    "map_html": map_path,
    "matrix_png": fig_path,
    "line_png": line_path,
    "env": {"python": sys.version.split()[0], "geopandas": gpd.__version__, "folium": folium.__version__}
}
TABLES = {"regions_monthly_change": table_regions, "summary_metrics": metrics}
MAP_OBJ = m
FIGS = []
NOTES = ["Expected dip in Apr–May (COVID) vs Jan baseline."]
'''


def code_template_hazardous() -> str:
    return r'''
# === Hazardous Waste × NC tracts ===
import sys
import numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt
import folium
from folium.plugins import Fullscreen, MiniMap

hw_url = "https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/HW_Sites_EPSG4326.zip"
tr_url = "https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_37_EPSG4326.zip"
pop_url = "https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv"

hw = gpd.read_file(hw_url).to_crs("EPSG:4326")
tr = gpd.read_file(tr_url).to_crs("EPSG:4326")
pop = pd.read_csv(pop_url, dtype={"GEOID": str})
tr["GEOID"] = tr["GEOID"].astype(str)
tr = tr.merge(pop, on="GEOID", how="left").dropna(subset=["TotalPopulation"])
hits = gpd.sjoin(tr, hw, how="inner", predicate="intersects").drop_duplicates(subset=["GEOID"])

# ===== Map — center bằng total_bounds
minx, miny, maxx, maxy = tr.total_bounds
center_lat, center_lon = (miny+maxy)/2.0, (minx+maxx)/2.0
m = folium.Map(location=[center_lat, center_lon], zoom_start=7, tiles="CartoDB positron")
Fullscreen().add_to(m); MiniMap(toggle_display=True, minimized=True).add_to(m)

folium.Choropleth(
    geo_data=tr.__geo_interface__, name="Total Population",
    data=tr[["GEOID","TotalPopulation"]], columns=["GEOID","TotalPopulation"],
    key_on="feature.properties.GEOID",
    fill_color="Blues", fill_opacity=0.7, line_opacity=0.2,
    legend_name="Total Population"
).add_to(m)

folium.GeoJson(
    hits.__geo_interface__, name="Tracts with Hazardous Waste",
    style_function=lambda f: {"color":"red","fillColor":"none","weight":2},
    tooltip=folium.GeoJsonTooltip(fields=["GEOID"], aliases=["GEOID:"], sticky=False)
).add_to(m)

folium.LayerControl().add_to(m)
map_path = "nc_hazardous_map.html"; m.save(map_path)

# ===== Simple chart: population histogram
plt.figure(figsize=(8,4))
tr["TotalPopulation"].plot(kind="hist", bins=30, alpha=0.7)
plt.title("Distribution of Tract Population (NC)")
plt.xlabel("Population"); plt.ylabel("Frequency")
hist_path = "nc_population_hist.png"
plt.savefig(hist_path, dpi=160, bbox_inches="tight"); plt.close()

# ===== Tables
table_hits = hits[["GEOID"]].copy()
table_pop_top = tr[["GEOID","TotalPopulation"]].sort_values("TotalPopulation", ascending=False).head(10)
metrics = pd.DataFrame({"metric":["num_hw_tracts","num_tracts"],
                        "value":[int(len(hits)), int(len(tr))]})

ARTIFACTS = {"map_html": map_path, "hist_png": hist_path,
             "env": {"python": sys.version.split()[0], "geopandas": gpd.__version__, "folium": folium.__version__}}
TABLES = {"hw_hits": table_hits, "top10_population": table_pop_top, "summary_metrics": metrics}
MAP_OBJ = m
FIGS = []
NOTES = ["HW tracts highlighted in red."]
'''


def code_template_covid() -> str:
    return r'''
# === COVID-19 2020 × US counties ===
import sys
import numpy as np, pandas as pd, geopandas as gpd, matplotlib.pyplot as plt
import folium
from folium.plugins import Fullscreen, MiniMap
from scipy import stats

covid_url = "https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv"
cnty_url  = "https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip"
cen_url   = "https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv"

covid = pd.read_csv(covid_url)
covid["date"] = pd.to_datetime(covid["date"], errors="coerce")
latest = covid[covid["date"]=="2020-12-31"].dropna(subset=["fips"]).copy()
latest["fips"] = latest["fips"].astype(int).astype(str).str.zfill(5)
cnty = gpd.read_file(cnty_url); cnty["GEOID"] = cnty["GEOID"].astype(str).str.zfill(5)
cen  = pd.read_csv(cen_url);  cen["FIPS"]  = cen["FIPS"].astype(str).str.zfill(5)

latest["death_rate"] = latest.apply(lambda r:(r["deaths"]/r["cases"]*100) if r["cases"]>0 else 0, axis=1)

sen_cols = [c for c in ["Total Population: 65 to 74 Years","Total Population: 75 to 84 Years","Total Population: 85 Years and Over"] if c in cen.columns]
def _to_float(s):
    try:
        return float(str(s).replace(",",""))
    except Exception:
        return np.nan
for _c in ["Total Population"] + sen_cols:
    if _c in cen.columns:
        cen[_c] = cen[_c].map(_to_float)

cen["senior_population"] = cen[sen_cols].sum(axis=1)
cen["senior_rate"] = (cen["senior_population"]/cen["Total Population"]*100).replace([np.inf,-np.inf],0).fillna(0)

merged    = latest.merge(cen, left_on="fips", right_on="FIPS", how="inner")
cnty_full = cnty.merge(merged, left_on="GEOID", right_on="fips", how="inner")

# ===== Scatter + regression
x = pd.to_numeric(cnty_full["senior_rate"], errors="coerce").astype(float).values
y = pd.to_numeric(cnty_full["death_rate"], errors="coerce").astype(float).values
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
xp = np.array([np.nanmin(x), np.nanmax(x)]); yp = slope*xp + intercept
plt.figure(figsize=(8,6))
plt.scatter(x, y, alpha=0.4, label="counties")
plt.plot(xp, yp, "r", label=f"fit (r²={r_value**2:.3f}, p={p_value:.4f})")
plt.xlabel("Senior rate (%)"); plt.ylabel("COVID-19 Death rate (%)")
plt.title("County-level: Seniors vs COVID-19 Death Rate (2020)")
plt.grid(True, alpha=0.3); plt.legend()
scatter_path = "covid_senior_scatter.png"
plt.savefig(scatter_path, dpi=220, bbox_inches="tight"); plt.close()

# ===== Map — làm sạch datetime/kiểu số cho Folium; center bằng total_bounds
d = cnty_full.to_crs("EPSG:4326").copy()
d["GEOID"] = d["GEOID"].astype(str)

# a) datetime -> chuỗi
for c in d.select_dtypes(include=["datetime64[ns]", "datetimetz"]).columns:
    d[c] = d[c].dt.strftime("%Y-%m-%d")

# b) numeric numpy -> float thuần
num_like = d.select_dtypes(include=["float64","float32","int64","int32","uint8","uint16","uint32","uint64"]).columns
for c in num_like:
    d[c] = pd.to_numeric(d[c], errors="coerce").astype(float)

# c) bỏ cột không dùng để tránh payload lớn
drop_cols = [c for c in ["date"] if c in d.columns]
d_vis = d.drop(columns=drop_cols) if drop_cols else d

# center an toàn
minx, miny, maxx, maxy = d_vis.total_bounds
center_lat, center_lon = (miny+maxy)/2.0, (minx+maxx)/2.0
m = folium.Map(location=[center_lat, center_lon], zoom_start=5, tiles="CartoDB positron")
Fullscreen().add_to(m); MiniMap(toggle_display=True, minimized=True).add_to(m)

chorodata = d_vis[["GEOID","death_rate"]].copy()
folium.Choropleth(
    geo_data=d_vis.to_json(),
    data=chorodata, columns=["GEOID","death_rate"],
    key_on="feature.properties.GEOID",
    fill_color="YlOrRd", fill_opacity=0.7, line_opacity=0.2,
    legend_name="COVID-19 Death rate (%)"
).add_to(m)

tooltip_fields = [c for c in ["NAME","GEOID","cases","deaths","death_rate","senior_rate"] if c in d_vis.columns]
tooltip_alias  = ["County","FIPS","Cases","Deaths","Death rate (%)","Senior rate (%)"][:len(tooltip_fields)]
folium.GeoJson(
    d_vis.to_json(),
    name="Labels",
    tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_alias, localize=True, labels=True, sticky=False)
).add_to(m)

folium.LayerControl().add_to(m)
map_path = "covid_death_rate_map.html"; m.save(map_path)

# ===== Tables & metrics
metrics = pd.DataFrame({"metric":["r2","pvalue","num_counties"],
                        "value":[float(r_value**2), float(p_value), int(len(d_vis))]})
top10 = d_vis[["GEOID","death_rate"]].sort_values("death_rate", ascending=False).head(10)

ARTIFACTS = {"map_html": map_path, "scatter_png": scatter_path,
             "r2": float(r_value**2), "p": float(p_value),
             "env": {"python": sys.version.split()[0], "geopandas": gpd.__version__, "folium": folium.__version__}}
TABLES = {"top10_death_rate": top10, "summary_metrics": metrics}
MAP_OBJ = m
FIGS = []
NOTES = [f"Correlation seniors→death: R²={float(r_value**2):.3f}, p={float(p_value):.4f}"]
'''

In [9]:
class RunState(TypedDict, total=False):
    system: str
    case: str
    seed: int
    max_retries: int

    _code: Optional[str]
    artifacts: Dict[str, Any]
    _tables: Dict[str, pd.DataFrame]
    _map_obj: Optional[folium.Map]
    _figs: List[Any]
    _notes: List[str]

    success: int
    runtime_sec: float
    errors_occurred: int
    errors_recovered: int
    validator_failed: int
    traceback_short: str
    api_calls: int
    retries: int
    prompt_tokens: int
    completion_tokens: int
    artifacts_ok: int


def _run_generated(code: str) -> Dict[str, Any]:
    import os, time, random, glob
    glb = {
        'np': np, 'pd': pd, 'gpd': gpd, 'plt': plt, 'folium': folium,
        'Fullscreen': Fullscreen, 'MiniMap': MiniMap,
        'box': box, 'mapping': mapping, 'calendar': calendar,
        'try_get': try_get, 'StringIO': StringIO,

        # >>> add these
        'requests': requests,
        'os': os,
        'time': time,
        'random': random,
        'glob': glob,
        # (optional) also expose download_file if you defined it in notebook scope
        'download_file': globals().get('download_file', None),
    }
    loc = {}
    exec(code, glb, loc)
    return {
        'ARTIFACTS': loc.get('ARTIFACTS', {}),
        'TABLES': loc.get('TABLES', {}),
        'MAP_OBJ': loc.get('MAP_OBJ', None),
        'FIGS': loc.get('FIGS', []),
        'NOTES': loc.get('NOTES', []),
    }


def executor(state: RunState) -> RunState:
    t0 = time.time()
    np.random.seed(int(state.get('seed', 0)))
    try:
        out = _run_generated(state.get('_code') or '')
        state['errors_occurred'] = 0
        state['traceback_short'] = ''
        state['artifacts'] = out['ARTIFACTS']
        state['_tables'] = out['TABLES']
        state['_map_obj'] = out['MAP_OBJ']
        state['_figs'] = out['FIGS']
        state['_notes'] = out['NOTES']
    except Exception as e:
        tb = traceback.format_exc()
        state['errors_occurred'] = 1
        state['traceback_short'] = ''.join(tb.splitlines()[-30:])
        state['artifacts'] = {}
        state['_tables'] = {}
        state['_map_obj'] = None
        state['_figs'] = []
        state['_notes'] = [f'error: {e}']
    state['runtime_sec'] = float(time.time() - t0)
    return state


def validator(state: RunState) -> RunState:
    # If executor already failed, mark validator failed too
    if int(state.get("errors_occurred", 0)) == 1:
        state["artifacts_ok"] = 0
        state["validator_failed"] = 1
        return state

    ok = artifacts_ok(state['case'])
    state['artifacts_ok'] = int(ok)
    state['validator_failed'] = int(not ok)
    return state

In [10]:
SYSTEM_PROMPT_CODER = (
    "You are a senior Python GIS engineer. Output ONLY runnable Python code. "
    "Do not wrap in markdown fences. "
    "The code MUST define variables ARTIFACTS, TABLES, MAP_OBJ, FIGS, NOTES. "
    "The code MUST save required artifacts with exact filenames."
)
SYSTEM_PROMPT_DEBUGGER = (
    "You are a Python debugger. Given failing code and traceback, output a patched full code. "
    "Output ONLY code, no markdown."
)

def required_files_for_case(case: str) -> List[str]:
    return CASE_SPECS[case]['required_files']

def _case_data_urls(case: str) -> str:
    if case == 'france_mobility':
        return (
            "France polygons: https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip"
            "Mobility API: http://gis.cas.sc.edu/GeoAnalytics/REST?operation=get_daily_movement_for_all_places&source=twitter&scale=world_first_level_admin&begin=01/01/2020&end=12/31/2020"
        )
    if case == 'hazardous_nc':
        return (
            "HW sites: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/HW_Sites_EPSG4326.zip"
            "NC tracts: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/tract_37_EPSG4326.zip"
            "Population: https://github.com/gladcolor/LLM-Geo/raw/master/overlay_analysis/NC_tract_population.csv"
        )
    if case == 'covid_us':
        return (
            "NYTimes COVID: https://github.com/nytimes/covid-19-data/raw/master/us-counties-2020.csv"
            "Counties: https://github.com/gladcolor/spatial_data/raw/master/contiguous_counties.zip"
            "ACS2020 county: https://raw.githubusercontent.com/gladcolor/spatial_data/master/Demography/ACS2020_5year_county.csv"
        )
    return ""

def _strip_code_fences(s: str) -> str:
    s = (s or '').strip()
    if s.startswith('```'):
        s = re.sub(r'^```[a-zA-Z0-9_-]*', '', s)
        s = re.sub(r'```\s*$', '', s)
    return s.strip()

def coder(case: str, seed: int) -> str:
    req_list = required_files_for_case(case)
    req = "\n".join(f"- {f}" for f in req_list)

    extra_france = ""
    if case == "france_mobility":
        extra_france = r"""
FRANCE_MOBILITY DATA LOADING (MANDATORY):
1) You MUST download France.zip locally (do NOT use /vsizip/France.zip unless you created it locally).
   Example:
     import os
     from pathlib import Path
     url = "https://github.com/gladcolor/LLM-Geo/raw/master/REST_API/France.zip"
     local_zip = "France.zip"
     # use requests with retry or the provided download_file(url, local_zip)
2) Then load boundaries via GeoPandas using zip://
   Example:
     import geopandas as gpd
     gdf = gpd.read_file(f"zip://{local_zip}")
3) Mobility API may timeout: implement retry/backoff OR fallback to synthetic data,
   but you MUST still produce the 3 required artifacts (if using synthetic, mention in NOTES).
4) You MUST NOT "skip data processing" without raising an exception.
"""

    prompt = f"""\
Task: Implement the GIS analysis for case={case}.

ABSOLUTE REQUIREMENTS:
A) You MUST save these exact artifact filenames in the CURRENT WORKING DIRECTORY:
{req}

B) You MUST define these variables:
- ARTIFACTS: dict
- TABLES: dict[str, pandas.DataFrame]
- MAP_OBJ: folium.Map or None
- FIGS: list
- NOTES: list[str]

C) You MUST set ARTIFACTS to reference the exact filenames above (strings).

D) At the very end of the script, you MUST verify files exist and FAIL-FAST:
import os
missing = [f for f in {req_list!r} if not os.path.exists(f)]
if missing:
    raise RuntimeError("Missing required artifacts: " + str(missing))

E) Do NOT print markdown. Output ONLY runnable Python code.

Data URLs:
{_case_data_urls(case)}

{extra_france}

Return only Python code.
"""
    code, _ = call_llm(SYSTEM_PROMPT_CODER, prompt)
    return _strip_code_fences(code)


In [11]:
# ===== PATCH 1: fix generate_code_for_system() unreachable code bug =====
def generate_code_for_system(system: str, case: str, seed: int) -> str:
    k = (system, case)

    # Rule-GIS always uses deterministic templates (no LLM)
    if system == 'Rule-GIS':
        if case == 'france_mobility':
            code = code_template_france()
        elif case == 'hazardous_nc':
            code = code_template_hazardous()
        elif case == 'covid_us':
            code = code_template_covid()
        else:
            raise ValueError(case)
        CODE_CACHE[k] = code
        return code

    # For LLM systems, we REQUIRE warmup to have populated cache once per (system, case)
    if k not in CODE_CACHE:
        raise RuntimeError(f"Missing cached code for {k}. Run warmup_codegen_gemini_once() first.")
    return CODE_CACHE[k]

In [12]:
def warmup_codegen(systems=None, cases=None):
    systems = systems or SYSTEMS
    cases = cases or CASES

    for system in systems:
        if system == "Rule-GIS":
            continue
        for case in cases:
            k = (system, case)
            if k in CODE_CACHE:
                print(f"[warmup] cache hit {k}")
                continue

            # Force template to avoid LLM + quota issues
            if case == "france_mobility":
                CODE_CACHE[k] = code_template_france()
            elif case == "hazardous_nc":
                CODE_CACHE[k] = code_template_hazardous()
            elif case == "covid_us":
                CODE_CACHE[k] = code_template_covid()
            else:
                raise ValueError(case)

            print(f"[warmup] template cached for {k}")

In [13]:
# ===== PATCH: Make ENABLE_LLM_DEBUGGER actually work =====
def run_one(system: str, case: str, seed: int) -> Dict[str, Any]:
    before_calls = USAGE.api_calls
    before_pt = USAGE.prompt_tokens
    before_ct = USAGE.completion_tokens

    max_r = MAX_RETRIES_MAGEOAI if system == 'MA-GeoAI' else MAX_RETRIES_TOOLCALL

    st: RunState = {
        'system': system,
        'case': case,
        'seed': int(seed),
        'max_retries': int(max_r),
        'success': 0,
        'errors_occurred': 0,
        'errors_recovered': 0,
        'validator_failed': 0,
        'traceback_short': '',
        'retries': 0,
    }

    st['_code'] = generate_code_for_system(system, case, seed)

    for r in range(int(max_r) + 1):
        st['retries'] = r
        st = executor(st)
        st = validator(st)

        # IMPORTANT: if validator fails, count it as an error (paper-grade ERR/TSR)
        if int(st.get("validator_failed", 0)) == 1:
            st["errors_occurred"] = 1

        ok = (st.get('errors_occurred', 0) == 0) and (st.get('validator_failed', 0) == 0)
        if ok:
            st['success'] = 1
            break

        # ==== Debugger branch guarded ====
        if (r < int(max_r)) and (system != 'Rule-GIS') and (globals().get("ENABLE_LLM_DEBUGGER", False) is True):
            st['errors_recovered'] = 1
            patched = debugger_patch(case, st.get('_code') or '', st.get('traceback_short') or '')
            st['_code'] = patched
            CODE_CACHE[(system, case)] = patched
        else:
            # no debugger: stop early after first failure (optional)
            # break
            pass

    st['api_calls'] = USAGE.api_calls - before_calls
    st['prompt_tokens'] = USAGE.prompt_tokens - before_pt
    st['completion_tokens'] = USAGE.completion_tokens - before_ct

    return dict(st)


In [14]:
# === PAPER-READY (clean) runner: codegen once per (system,case), no debugger, robust exec, optional fallback ===
import os, time, random, traceback
import pandas as pd


# ---- CSV append helper (self-contained; safe if cell 14 not run) ----
from pathlib import Path

def _append_df_to_csv(df: pd.DataFrame, csv_path: Path):
    csv_path = Path(csv_path)
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    if csv_path.exists():
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, index=False)


ENABLE_LLM_DEBUGGER = False
ENABLE_GEMINI_CODEGEN = True
LLM_SYSTEMS = ["Single-Agent", "ToolCall+Retries", "AutoGen-MAS-sim", "MA-GeoAI"]

# ---- robust small GET helper (text) for generated code if needed ----
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def _requests_session_paper():
    s = requests.Session()
    retry = Retry(total=4, connect=4, read=4, backoff_factor=0.8,
                  status_forcelist=[429, 500, 502, 503, 504], allowed_methods=["GET", "HEAD"])
    s.mount("http://", HTTPAdapter(max_retries=retry))
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

def safe_get_text(url, timeout=(5, 25), tries=3):
    sess = _requests_session_paper()
    last = None
    for i in range(tries):
        try:
            r = sess.get(url, timeout=timeout)
            r.raise_for_status()
            return r.text
        except Exception as e:
            last = e
            time.sleep((2 ** i) + random.uniform(0, 0.5))
    raise RuntimeError(f"safe_get_text failed after {tries} tries: {last}")

# ---- executor with full globals for generated code ----

def _run_generated_paper(code: str):
    import glob, calendar
    glb = {
        'np': np, 'pd': pd, 'gpd': gpd, 'plt': plt, 'folium': folium,
        'Fullscreen': Fullscreen, 'MiniMap': MiniMap,
        'box': box, 'mapping': mapping, 'calendar': calendar,
        'StringIO': StringIO,
        'requests': requests, 'os': os, 'time': time, 'random': random, 'glob': glob,
        'safe_get_text': safe_get_text,
        'try_get': try_get,
        'download_file': globals().get('download_file', None),
    }
    loc = {}
    exec(code, glb, loc)
    return {
        'ARTIFACTS': loc.get('ARTIFACTS', {}),
        'TABLES': loc.get('TABLES', {}),
        'MAP_OBJ': loc.get('MAP_OBJ', None),
        'FIGS': loc.get('FIGS', []),
        'NOTES': loc.get('NOTES', []),
    }


def executor_paper(state: dict) -> dict:
    import socket
    socket.setdefaulttimeout(10)  # hard cap for any network call in generated code
    t0 = time.time()
    np.random.seed(int(state.get('seed', 0)))
    try:
        out = _run_generated_paper(state.get('_code') or '')
        state['errors_occurred'] = 0
        state['traceback_short'] = ''
        state['artifacts'] = out['ARTIFACTS']
        state['_tables'] = out['TABLES']
        state['_map_obj'] = out['MAP_OBJ']
        state['_figs'] = out['FIGS']
        state['_notes'] = out['NOTES']
    except Exception:
        tb = traceback.format_exc()
        state['errors_occurred'] = 1
        state['traceback_short'] = "".join(tb.splitlines()[-35:])
        state['artifacts'] = {}
        state['_tables'] = {}
        state['_map_obj'] = None
        state['_figs'] = []
        state['_notes'] = ['executor error']
    state['runtime_sec'] = float(time.time() - t0)
    return state


def validator_paper(state: dict) -> dict:
    if int(state.get('errors_occurred', 0)) == 1:
        state['artifacts_ok'] = 0
        state['validator_failed'] = 1
        return state
    ok = artifacts_ok(state['case'])
    state['artifacts_ok'] = int(ok)
    state['validator_failed'] = int(not ok)
    return state


def template_for_case(case: str) -> str:
    if case == 'france_mobility':
        return code_template_france()
    if case == 'hazardous_nc':
        return code_template_hazardous()
    if case == 'covid_us':
        return code_template_covid()
    raise ValueError(case)

# ---- warmup: codegen once per (system,case), record cost ----
CODEGEN_COST = {}

def warmup_codegen_gemini_once_paper(systems=None, cases=None):
    systems = systems or SYSTEMS
    cases = cases or CASES
    for system in systems:
        for case in cases:
            k = (system, case)
            if k in CODE_CACHE:
                print(f"[warmup] cache hit {k}")
                continue

            # ---- FORCE TEMPLATE for france_mobility (fast + reproducible; avoid gis.cas.sc.edu) ----
            if case == "france_mobility":
                CODE_CACHE[k] = template_for_case(case)  # template already creates required artifacts
                print(f"[warmup] FORCE TEMPLATE (no external API) for {k}")
                continue

            if system == 'Rule-GIS':
                CODE_CACHE[k] = template_for_case(case)
                print(f"[warmup] template cached for {k}")
                continue

            if ENABLE_GEMINI_CODEGEN and system in LLM_SYSTEMS:
                b_calls, b_pt, b_ct = USAGE.api_calls, USAGE.prompt_tokens, USAGE.completion_tokens
                print(f"[warmup] GEMINI codegen for {k} (one-time)")
                CODE_CACHE[k] = coder(case, seed=0)
                dcalls = USAGE.api_calls - b_calls
                dpt = USAGE.prompt_tokens - b_pt
                dct = USAGE.completion_tokens - b_ct
                CODEGEN_COST[k] = {'api_calls': dcalls, 'prompt_tokens': dpt, 'completion_tokens': dct}
                print(f"[warmup] GEMINI usage {k}: api_calls +{dcalls}, prompt_tokens +{dpt}, completion_tokens +{dct}")
            else:
                CODE_CACHE[k] = template_for_case(case)
                print(f"[warmup] template cached for {k}")


# ---- persist required artifacts into ARTIFACT_DIR ----
REQUIRED_ARTIFACTS = [
    'france_mobility_map_matrix.png',
    'france_mobility_line.png',
    'france_mobility_dec2020.html',
    'nc_hazardous_map.html',
    'nc_population_hist.png',
    'covid_death_rate_map.html',
    'covid_senior_scatter.png',
]

def sync_artifacts_to_drive(work_dir='/content'):
    work_dir = Path(work_dir)
    for name in REQUIRED_ARTIFACTS:
        src = work_dir / name
        if src.exists():
            shutil.copy2(src, ARTIFACT_DIR / name)

def run_one_paper(system: str, case: str, seed: int, max_retries: int = 2, fallback_to_template: bool = True) -> dict:
    before_calls, before_pt, before_ct = USAGE.api_calls, USAGE.prompt_tokens, USAGE.completion_tokens

    st = {
        'system': system, 'case': case, 'seed': int(seed),
        'success': 0, 'errors_occurred': 0, 'errors_recovered': 0,
        'validator_failed': 0, 'artifacts_ok': 0,
        'traceback_short': '', 'retries': 0,
        'fallback_used': 0,
    }

    k = (system, case)
    if k not in CODE_CACHE:
        raise RuntimeError(f"Missing cached code for {k}. Run warmup_codegen_gemini_once_paper first.")
    st['_code'] = CODE_CACHE[k]

    for r in range(max_retries + 1):
        st['retries'] = r
        st = executor_paper(st)
        st = validator_paper(st)
        if int(st.get('validator_failed', 0)) == 1:
            st['errors_occurred'] = 1
        if st['errors_occurred'] == 0 and st['validator_failed'] == 0:
            st['success'] = 1
            break

    if st['success'] == 0 and fallback_to_template and system != 'Rule-GIS':
        st['fallback_used'] = 1
        st['_code'] = template_for_case(case)
        st = executor_paper(st)
        st = validator_paper(st)
        if st['errors_occurred'] == 0 and st['validator_failed'] == 0:
            st['success'] = 1
            st['errors_recovered'] = 1

    st['api_calls'] = USAGE.api_calls - before_calls
    st['prompt_tokens'] = USAGE.prompt_tokens - before_pt
    st['completion_tokens'] = USAGE.completion_tokens - before_ct
    return st


def run_grid_two_phase_paid_paper(systems=None, cases=None, seeds=None, fallback_to_template: bool = True):
    systems = systems or SYSTEMS
    cases = cases or CASES
    seeds = seeds or SEEDS

    # ---- persistent run id & output files (avoid overwrite) ----
    run_id = time.strftime('%Y%m%d_%H%M%S')
    runs_csv = LOG_DIR / f'runs_{run_id}.csv'
    cost_csv = LOG_DIR / f'codegen_cost_{run_id}.csv'
    # also maintain latest pointers
    latest_runs_csv = RUNS_CSV_PATH
    latest_cost_csv = COST_CSV_PATH

    print('[run_id]', run_id)
    print('runs_csv:', runs_csv)
    print('cost_csv:', cost_csv)

    warmup_codegen_gemini_once_paper(systems=systems, cases=cases)

    # incremental logging to avoid losing results on crash
    rows_written = 0
    for system in systems:
        for case in cases:
            for seed in seeds:
                print(f"== RUN system={system} case={case} seed={seed} ==")
                st_row = run_one_paper(system, case, seed, max_retries=0, fallback_to_template=fallback_to_template)
                # save row immediately (timestamped + latest)
                row_df = pd.DataFrame([st_row])
                _append_df_to_csv(row_df, runs_csv)
                _append_df_to_csv(row_df, latest_runs_csv)
                rows_written += 1

                sync_artifacts_to_drive('/content')

    df = pd.read_csv(runs_csv)
    print('[saved] rows:', len(df), 'to', runs_csv)

    if CODEGEN_COST:
        dfc = pd.DataFrame([{'system':k[0],'case':k[1], **v} for k,v in CODEGEN_COST.items()])
        dfc.to_csv(cost_csv, index=False)
        dfc.to_csv(latest_cost_csv, index=False)

    print('Saved runs with', len(df), 'rows; latest ->', latest_runs_csv)
    return df

print('[PAPER-READY] Use: df = run_grid_two_phase_paid_paper()')


[PAPER-READY] Use: df = run_grid_two_phase_paid_paper()


In [15]:
import pandas as pd

def _append_df_to_csv(df: pd.DataFrame, csv_path: Path):
    csv_path = Path(csv_path)
    csv_path.parent.mkdir(parents=True, exist_ok=True)
    if csv_path.exists():
        df.to_csv(csv_path, mode="a", header=False, index=False)
    else:
        df.to_csv(csv_path, index=False)

def save_runs_df(runs_df: pd.DataFrame):
    _append_df_to_csv(runs_df, RUNS_CSV_PATH)

def save_cost_df(cost_df: pd.DataFrame):

    cost_df.to_csv(COST_CSV_PATH, index=False)


In [16]:
CODE_CACHE = {}
df = run_grid_two_phase_paid_paper(
    systems=["Rule-GIS", "Single-Agent", "ToolCall+Retries", "AutoGen-MAS-sim", "MA-GeoAI"],
    cases=["france_mobility","hazardous_nc","covid_us"],
    seeds=list(range(10)),
    fallback_to_template=True
)

[run_id] 20260131_171928
runs_csv: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs/logs/runs_20260131_171928.csv
cost_csv: /content/drive/MyDrive/AI/langgraph_multi_agent/outputs/logs/codegen_cost_20260131_171928.csv
[warmup] FORCE TEMPLATE (no external API) for ('Rule-GIS', 'france_mobility')
[warmup] template cached for ('Rule-GIS', 'hazardous_nc')
[warmup] template cached for ('Rule-GIS', 'covid_us')
[warmup] FORCE TEMPLATE (no external API) for ('Single-Agent', 'france_mobility')
[warmup] GEMINI codegen for ('Single-Agent', 'hazardous_nc') (one-time)
[warmup] GEMINI usage ('Single-Agent', 'hazardous_nc'): api_calls +1, prompt_tokens +335, completion_tokens +1418
[warmup] GEMINI codegen for ('Single-Agent', 'covid_us') (one-time)
[warmup] GEMINI usage ('Single-Agent', 'covid_us'): api_calls +1, prompt_tokens +336, completion_tokens +1649
[warmup] FORCE TEMPLATE (no external API) for ('ToolCall+Retries', 'france_mobility')
[warmup] GEMINI codegen for ('ToolCall+Retries', 'haz

In [17]:
import pandas as pd
df = pd.read_csv(RUNS_CSV_PATH)
print("rows:", len(df))
print(df.groupby(["system","case"])["seed"].nunique().min(), "seeds min per (system,case)")


rows: 298
10 seeds min per (system,case)


In [18]:
import pandas as pd
summary = df.groupby(["system","case"]).agg(
    TSR=("success","mean"),
    ERR=("errors_occurred","mean"),
    ValidatorFail=("validator_failed","mean"),
    RuntimeMean=("runtime_sec","mean"),
    FallbackRate=("fallback_used","mean"),
    APIcalls=("api_calls","sum"),
    PromptTokens=("prompt_tokens","sum"),
    CompletionTokens=("completion_tokens","sum"),
).reset_index()
summary


,system,case,TSR,ERR,ValidatorFail,RuntimeMean,FallbackRate,APIcalls,PromptTokens,CompletionTokens
0,AutoGen-MAS-sim,covid_us,1.0,0.0,0.0,14.236398,1.000000,0,0,0
1,AutoGen-MAS-sim,france_mobility,1.0,0.0,0.0,12.693328,0.473684,0,0,0
2,AutoGen-MAS-sim,hazardous_nc,1.0,0.0,0.0,28.214044,1.000000,0,0,0
3,MA-GeoAI,covid_us,1.0,0.0,0.0,14.495864,1.000000,0,0,0
4,MA-GeoAI,france_mobility,1.0,0.0,0.0,10.821120,0.000000,0,0,0
5,MA-GeoAI,hazardous_nc,1.0,0.0,0.0,29.246468,1.000000,0,0,0
6,Rule-GIS,covid_us,1.0,0.0,0.0,16.078327,0.000000,0,0,0
7,Rule-GIS,france_mobility,1.0,0.0,0.0,10.518572,0.000000,0,0,0
8,Rule-GIS,hazardous_nc,1.0,0.0,0.0,30.874858,0.000000,0,0,0
9,Single-Agent,covid_us,1.0,0.0,0.0,16.828233,1.000000,0,0,0


In [19]:
import pandas as pd
df = pd.read_csv(RUNS_CSV_PATH)

summary = df.groupby(["system","case"]).agg(
    TSR=("success","mean"),
    ERR=("errors_occurred","mean"),
    ValidatorFail=("validator_failed","mean"),
    RuntimeMean=("runtime_sec","mean"),
    RuntimeStd=("runtime_sec","std"),
    FallbackRate=("fallback_used","mean"),
    N=("success","count"),
).reset_index()

summary


,system,case,TSR,ERR,ValidatorFail,RuntimeMean,RuntimeStd,FallbackRate,N
0,AutoGen-MAS-sim,covid_us,1.0,0.0,0.0,14.236398,1.613945,1.000000,10
1,AutoGen-MAS-sim,france_mobility,1.0,0.0,0.0,12.693328,2.956945,0.473684,19
2,AutoGen-MAS-sim,hazardous_nc,1.0,0.0,0.0,28.214044,1.875494,1.000000,10
3,MA-GeoAI,covid_us,1.0,0.0,0.0,14.495864,1.866306,1.000000,10
4,MA-GeoAI,france_mobility,1.0,0.0,0.0,10.821120,1.607900,0.000000,10
5,MA-GeoAI,hazardous_nc,1.0,0.0,0.0,29.246468,2.419457,1.000000,10
6,Rule-GIS,covid_us,1.0,0.0,0.0,16.078327,1.900992,0.000000,20
7,Rule-GIS,france_mobility,1.0,0.0,0.0,10.518572,1.420193,0.000000,20
8,Rule-GIS,hazardous_nc,1.0,0.0,0.0,30.874858,2.295020,0.000000,20
9,Single-Agent,covid_us,1.0,0.0,0.0,16.828233,2.047786,1.000000,33


In [20]:
import numpy as np
summary_ci = df.groupby(["system","case"]).apply(
    lambda g: pd.Series({
        "RuntimeMean": g["runtime_sec"].mean(),
        "RuntimeCI95": 1.96 * g["runtime_sec"].std(ddof=1) / np.sqrt(len(g)),
        "TSR": g["success"].mean(),
        "ERR": g["errors_occurred"].mean(),
        "FallbackRate": g["fallback_used"].mean(),
        "N": len(g),
    })
).reset_index()
summary_ci


/tmp/ipython-input-1446798243.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_ci = df.groupby(["system","case"]).apply(


,system,case,RuntimeMean,RuntimeCI95,TSR,ERR,FallbackRate,N
0,AutoGen-MAS-sim,covid_us,14.236398,1.000334,1.0,0.0,1.000000,10.0
1,AutoGen-MAS-sim,france_mobility,12.693328,1.329605,1.0,0.0,0.473684,19.0
2,AutoGen-MAS-sim,hazardous_nc,28.214044,1.162443,1.0,0.0,1.000000,10.0
3,MA-GeoAI,covid_us,14.495864,1.156749,1.0,0.0,1.000000,10.0
4,MA-GeoAI,france_mobility,10.821120,0.996587,1.0,0.0,0.000000,10.0
5,MA-GeoAI,hazardous_nc,29.246468,1.499595,1.0,0.0,1.000000,10.0
6,Rule-GIS,covid_us,16.078327,0.833147,1.0,0.0,0.000000,20.0
7,Rule-GIS,france_mobility,10.518572,0.622427,1.0,0.0,0.000000,20.0
8,Rule-GIS,hazardous_nc,30.874858,1.005837,1.0,0.0,0.000000,20.0
9,Single-Agent,covid_us,16.828233,0.698689,1.0,0.0,1.000000,33.0
